[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Neuvork/Engeneering-thesis/blob/master/results.ipynb)

In [ ]:
! git clone https://<username>:<password>@github.com/Neuvork/Engeneeringthesis.git --single-branch --branch cmaes

In [ ]:
#DOPISAC CMA
#ZROBIC REKURENCYJNY ES
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import sqrtm
import time
from IPython.display import clear_output
import copy
import cupy as cp
import tensorflow_datasets as tfds
import tensorflow as tf
from Engeneeringthesis.sigmas import Sigmas_Neural_Network
from Engeneeringthesis.NeuralNetwork import Neural_Network
from Engeneeringthesis.Cma_es import CMA_ES

In [ ]:
!nvidia-smi

In [ ]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
def cuda_memory_clear():
    mempool.free_all_blocks()
    pinned_mempool.free_all_blocks()          

In [ ]:
no_debug = 1
basic_debug_mode = 2
super_debug_mode = 3
only_interesting = 5
DEBUG_MODE = only_interesting

In [ ]:
train_ds_mnist = tfds.load("mnist", split = "train", shuffle_files=True, batch_size=-1)
test_ds_mnist = tfds.load("mnist", split = "test", shuffle_files=True, batch_size=-1)

train_ds_mnist = tfds.as_numpy(train_ds_mnist)
test_ds_mnist = tfds.as_numpy(test_ds_mnist)

train_ds_mnist = {"image" : cp.array(train_ds_mnist["image"]/255., dtype=cp.float32), "label" : cp.array(train_ds_mnist["label"]) }
test_ds_mnist = {"image" : cp.array(test_ds_mnist["image"]/255., dtype=cp.float32), "label" : cp.array(test_ds_mnist["label"]) }

In [ ]:
def evaluate_population(population, train_ds):
    create_input_time = 0
    preds_time = 0
    points_count_time = 0
    j  = 0
    if DEBUG_MODE % basic_debug_mode == 0:
      print("___EVALUATE_POPULATION_START")
    #scores = np.zeros(population.layers[0][1].shape[0], dtype = np.uint32)
    scores = cp.zeros(population.population_size, dtype = cp.uint32)
    for image, label in zip(cp.array(train_ds['image']), cp.array(train_ds['label'])):
        start = time.time()
        image = image.flatten()
        create_input_time += time.time() - start
        start = time.time()
        preds = population.forward(image)
        preds_time += time.time() - start
        start = time.time()
        #scores += cp.asnumpy(preds == label)
        scores += preds == label
        points_count_time += time.time() - start
        j += 1
      
    if DEBUG_MODE % basic_debug_mode == 0:
      print("___EVALUATE_POPULATION_STOP", "create_input_time: ", create_input_time, "preds_time:", preds_time,
          "points_count_time: ", points_count_time, "\n best result: ", np.max(cp.asnumpy( scores)),
          "mean socre: ", np.mean(cp.asnumpy( scores)), "min score: ", np.min(cp.asnumpy( scores))) 
    if DEBUG_MODE % only_interesting == 0:
      print("best result: ", np.max(cp.asnumpy( scores)), "mean socre: ", np.mean(cp.asnumpy( scores)), "min score: ", np.min(cp.asnumpy( scores)))

    return scores

In [ ]:
!nvidia-smi

In [ ]:
POPULATION_SIZE = 512
#input size do zmiany
population = Neural_Network(POPULATION_SIZE,  (28*28, 1, 1), [['linear', 10, [1.,1.]]])
sigmas = Sigmas_Neural_Network(POPULATION_SIZE,  (28*28, 1, 1), [['linear', 10, [1.,1.]]])

In [ ]:
classifier = CMA_ES(population, .05, evaluate_population)

In [ ]:
classifier.fit(train_ds_mnist, 256, 512, 500)